## Objectives

* We will tune the hyperparameters of for Logistic Regression and an Adaptive Boost model.

## Inputs

* Training and Testing data sets from notebook 04.
* Insights developed in the previous notebook.

## Outputs

* We will have saved models with tuned hyper parameters at the end of this notebook.

## Additional Comments

* We are making some philosophical assumptions about the nature of hyperparameters. The basic assumption is that the performance of a model trained with hyperparameters that are "near enough" to each other will perform "similarly enough." This is the idea that the performance of the model depends _continuously_ on the hyperparameters. We in fact assume a certain amount of regularity of this dependence. In partial differential equations (pdes), the kind of behavior we are assuming is characteristic of elliptic pdes. We do not have a technical reason for believing this. We assume this is an active area of research for various models, but in general it is outside the scope of this project. It does influence our decision in how we go about searching for good hyperparameters.

---
# Change working directory
We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()
import os

home_dir = '/workspace/pp5-ml-dashboard'
os.chdir(home_dir)
current_dir = os.getcwd()
print(current_dir)
We now load our training and test sets, as well as some of the packages that we will be using.
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from src.utils import get_df, save_df

train_dir = 'datasets/train/csv'
X_TrainSet = get_df('X_TrainSet',train_dir)
Y_TrainSet = get_df('Y_TrainSet',train_dir)

test_dir = 'datasets/test/csv'
X_TestSet = get_df('X_TestSet',test_dir)
Y_TestSet = get_df('Y_TestSet',test_dir)
## Section 1: Pipeline and Grid Search set up
We recall the code for building our pipelines and the grid search that we performed in the last notebook. Note that some of the constants have changed. 

We have modified the pipeline to see how feateure selection impacts the performance. Note that setting `thresh=1` essentially removes the `'corr_selector'` step from the pipeline. We will eventually remove this step from the pipeline once we have selected a value for `thresh`.
from sklearn.preprocessing import StandardScaler
from feature_engine import transformation as vt
from feature_engine.selection import DropFeatures, SmartCorrelatedSelection
from sklearn.pipeline import Pipeline


# Constants needed for feature engineering
TO_DROP = ['ftm_away', 'plus_minus_home', 'fg3m_away', 'pts_away', 'play_off',
           'fgm_away', 'pts_home', 'fg3m_home', 'ftm_home', 'fgm_home',
           'season']
THRESH = 0.6
TRANSFORMS = {'box_cox':(vt.BoxCoxTransformer,False),
              'yeo_johnson':(vt.YeoJohnsonTransformer,False)}
TRANSFORM_ASSIGNMENTS = {
    'yeo_johnson': ['dreb_away', 'blk_home', 'oreb_away', 'fta_away',
                    'dreb_home', 'ast_home', 'stl_away', 'stl_home',
                    'reb_away', 'oreb_home', 'pf_away', 'pf_home'],
    'box_cox': ['ast_away', 'fta_home']
                            }


def base_pipeline(thresh=THRESH):
    pipeline = Pipeline([
        ('dropper', DropFeatures(features_to_drop=TO_DROP)),
        ('corr_selector', SmartCorrelatedSelection(method="pearson",
                                                   threshold=thresh,
                                                   selection_method="variance")
                                                   )
                        ])
    return pipeline

    
def add_transformations(pipeline, transform_assignments):
    # This needs to be called after the above is fit so that the correlation selector has that attr
    dropping = pipeline['corr_selector'].features_to_drop_
    
    new_assignments = { key: [val for val in value if val not in dropping] 
                       for key,value in transform_assignments.items()}
    for transform, targets in new_assignments.items():
        if not targets:
            continue
        pipeline.steps.append(
            (transform, TRANSFORMS[transform][0](variables=targets))
            )
    pipeline.steps.append(('scaler', StandardScaler()))
    return pipeline
from sklearn.feature_selection import SelectFromModel

# ML algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier,AdaBoostClassifier


MODELS = {
    'LogisticRegression': LogisticRegression,
    'GradientBoosting': GradientBoostingClassifier,
    'AdaBoost': AdaBoostClassifier,
}

def create_pipe(model_name, random_state=42, params={}):
    model = MODELS[model_name](random_state=random_state,**params)
    base_pipe = base_pipeline()
    base_pipe.fit(X_TrainSet)
    pipe= add_transformations(base_pipe,TRANSFORM_ASSIGNMENTS)
    pipe.steps.append(("feat_selection", SelectFromModel(model)))
    pipe.steps.append(('model',model))
    pipe.model_type = model_name
    pipe.name = model_name
    return pipe

Next, we have the code for our grid search. As we will be treating `thresh` as a hyperparameter, it will be slightly different.
from sklearn.model_selection import GridSearchCV
# to suppress warnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import logging
logging.captureWarnings(True)
os.environ['PYTHONWARNINGS']='ignore'


def grid_search(X_train, y_train,pipe,param_grid={},verbosity=1):
    print(f"### Beginning grid search for {pipe.name} ###") 
    grid=GridSearchCV(estimator=pipe,
                    param_grid=param_grid,
                    cv=5,
                    n_jobs=-2,
                    verbose=verbosity,
                    scoring=['accuracy','precision'],
                    refit='precision')
    grid.fit(X_train,y_train)
    return grid

## Section 2: Logistic Regression
Logistic Regression models have many hyperparameters. We will focus on:
* `penalty` is a regularization parameter.
* `solver` specifies the type of algorithm used.
* `C` controls the strength of the penalty.

Not all penalties work for each solver.

These will be our initial choice for hyperparameters. They will help us narrow down our search and find other ranges to test.
thresholds = [round(0.1*i,2) for i in range(5,11)]
C = [10**(2*i+1) for i in range(-2,2)]
solver = ['newton-cg', 'newton-cholesky', 'lbfgs', 'liblinear', 'sag', 'saga']
penalty = ['none', 'l1', 'l2', 'elasticnet']
param_grid = [
            {'C':C,
             'solver':['lbfgs','newton-cg','newton-cholesky','sag'],
             'penalty':['l2',None]},
            {'C':C,
             'solver':['liblinear'],
             'penalty':['l1','l2']},
            {'C':C,
             'solver':['saga'],
             'penalty':['l1','l2',None,'elasticnet']}
              ]
logistic_param_grid = [{'model__'+key:value 
                                for key,value in param_dict.items()}
                                for param_dict in param_grid]
for params in logistic_param_grid:
    params['corr_selector__threshold']=thresholds

Now we are ready to do the grid search. We expect this to go well since Logistic Regression was our best performing model without any tuning. This initial training will help us establish a range to further tune the hyperparamters in.
from src.utils import save_df, get_df


def get_grid_results_df(pipe, name, dir, param_grid={}, verbosity=2):
    try:
        results_df = get_df(name, dir)
    except FileNotFoundError:
        pipe_grid_search = grid_search(
            X_TrainSet, Y_TrainSet, pipe, param_grid=param_grid, verbosity=verbosity
        )
        results_df = pd.DataFrame(pipe_grid_search.cv_results_)
        save_df(results_df, name, dir)
        # this normalizes the types
        results_df = get_df(name, dir)
    return results_df


logistic_pipe = create_pipe("LogisticRegression")
results_name = "logistic_grid_results_v1"
dir = "experiment_results/tuning/grids"

logistic_results_df_v1 = get_grid_results_df(
    logistic_pipe, results_name, dir, param_grid=logistic_param_grid
)

Let's look at what the best choices are at this stage.
from src.model_eval import get_best_params_df

get_best_params_df(logistic_results_df_v1)
Clearly, the best correlation threshold is 0.8. The models have the same scores. Lets collect all of the parameters that have the same scores. We order the scores not by count but by score.
from src.model_eval import present_score_counts, score_stats


present_score_counts(logistic_results_df_v1)
best_score = score_stats(logistic_results_df_v1)

The most common scores are `nan` (we changed the score to be -1 if a `nan` value was showing up). This happens when a choice of parameters does not work well together. We won't worry about this as we are able to get quite good precision and accuracy with this first pass. Note that the best accuracy score is not far from the accuracy of the model with the best precision.

34 different choices of parameters had the best performance. We would like to see what these estimators had in common and look at neighborhoods around these parameters to see if we can improve the performance before moving on to the next model.
from src.model_eval import present_param_counts

present_param_counts(logistic_results_df_v1, best_score)
We will make the following modifications to `logistic_param_grid`:
* remove `'liblinear'` and `'sag'` as they were used the least,
* pick a neighborhood around 0.8 for correlation threshold,
* focus on penalties `'l2'` and `'None'`
* focus on the range 1 to 1000 for `C`

We will see if focusing gives us any improvement in score.
from src.utils import divide_range

thresholds = divide_range(0.75,0.85,5)
C = divide_range(1,1000,6)
logistic_param_grid_v2=[{'model__C': C,
  'model__solver': ['lbfgs', 'newton-cg', 'newton-cholesky','saga'],
  'model__penalty': ['l2', None],
  'corr_selector__threshold': thresholds}]
logistic_results_df_v2 = get_grid_results_df(logistic_pipe,
                                             'logistic_results_df_v2', dir, 
                                             param_grid=logistic_param_grid_v2,
                                             verbosity=3)
Let's proceed by doing the analysis we did above of the results of this grid search.
get_best_params_df(logistic_results_df_v2)
Again, very similar scores. So let's analyze the scores that showed up as we did before.
present_score_counts(logistic_results_df_v2)
best_score = score_stats(logistic_results_df_v2)
Our most common score is our best score. It seems like we have chosen a good range of parameters since many of the combinations yield good results.
present_param_counts(logistic_results_df_v2, best_score)
All of the choices of parameters seem to be performing equally well, with insignificant exceptions. We will have to look at other metrics to determine distinguish between these choices of parameters. Things such as training time statistics, and standard deviation of the scores.
best_results = logistic_results_df_v2.query(f'mean_test_precision == {best_score[0]} and mean_test_accuracy == {best_score[1]}')
std_score_counts = {}

for _, row in best_results.iterrows():
    std_score = (row['std_test_precision'], row['std_test_accuracy'])
    if std_score in std_score_counts:
        std_score_counts[std_score] += 1
    else:
        std_score_counts[std_score] = 1
for key, value in std_score_counts.items():
    print(f"std_test_precision: {key[0]}"
          f"\nstd_test_accuracy: {key[1]}"
          f"\ncount: {value}")
So standard deviation will not help us distinguish either. This is annoying, but good. We will see how the models perform on the test data set.

Note: If you are tinkering and running cells multiple times, we recommend commenting out the code in the following three cells. They take a bit even after they have already been run the first time since we aren't saving the large number of pipelines and models.
import ast

def parameter_dicts(results_df, best_score):
    relevant = results_df.query(f'mean_test_precision == {best_score[0]} and mean_test_accuracy == {best_score[1]}')
    param_dicts = [ast.literal_eval(param_dict) for param_dict in relevant['params'].values]
    return param_dicts

best_params = parameter_dicts(logistic_results_df_v2, best_score)
'''
best_pipes = []
for param_dict in best_params:
    base_pipe = create_pipe('LogisticRegression')
    pipe = base_pipe.set_params(**param_dict)
    pipe.param_dict = param_dict
    best_pipes.append(pipe)

model_params = {key.split('__')[0]:key.split('__')[1]
                for key in best_params[0].keys()}
count = 0
for pipe in best_pipes:
    print(f"Pipe {count}:")
    for step in pipe.get_params()['steps']:
        if step[0] in model_params:
            param = model_params[step[0]]
            value = step[1].get_params()[param]
            print(f"{step[0]}"
                  f"\n{param}: {value}")
    print()
    count += 1
    if count>=2:
        break
'''
We will now train all of the above pipelines and evaluate them on the test dataset.
'''count = 0
for pipe in best_pipes:
    print(f"Training pipe {count}:")
    print(pipe.param_dict)
    pipe.fit(X_TrainSet, Y_TrainSet)
    count+=1
    '''
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

'''
def evaluate_param_on_test_set(pipe,X_test, Y_test):
    y_pred = pipe.predict(X_test)
    accuracy = accuracy_score(Y_test, y_pred)
    precision = precision_score(Y_test, y_pred)
    recall = recall_score(Y_test, y_pred)
    f1 = f1_score(Y_test, y_pred)
    return ((precision, accuracy, recall, f1), pipe.param_dict)


def evaluate_and_sort(fitted_pipes, X_test, Y_test):
    evaluations = [evaluate_param_on_test_set(pipe, X_test, Y_test)
               for pipe in fitted_pipes]
    evaluation_dict = {}
    for eval in evaluations:
        if eval[0] in evaluation_dict:
            evaluation_dict[eval[0]].append(eval[1])
        else:
            evaluation_dict[eval[0]] = [eval[1]]
    sorted_eval_dict = {k:v for k,v in sorted(evaluation_dict.items(),
                                         key=lambda item: item[0],
                                         reverse=True)}
    return sorted_eval_dict

sorted_evals = evaluate_and_sort(best_pipes, X_TestSet, Y_TestSet)
print(len(sorted_evals))
'''
It turns out that all of these best sets of parameters produce models that perform equally well with respect to the standard meterics. We will have to pick one to deploy. We will look at the time it took to score each model during the grid search.
time_results = best_results.filter(['mean_score_time','std_score_time','params'])

time_results = time_results.sort_values(by=['mean_score_time','std_score_time'])
print(time_results.head())
params_choice = time_results.iloc[0]['params']
print(params_choice)

We have the following choice of hyperparameters:
* correlation threshold: 0.77
* `C`: 500.5
* solver method: newton-cg
* penalty function: l2

Let's train the model and then look at the classification report. We don't need to list the penalty function since l2 is the default penalty function.
model_params = {'C':500.5, 'solver': 'newton-cg'}

final_logistic_pipe = create_pipe(model_name='LogisticRegression', params=model_params)
final_logistic_pipe.set_params(corr_selector__threshold=0.77)
final_logistic_pipe
Let's now see the importance of the different features according to our final model.
### Section 3: AdaBoost
We will now do a grid search with AdaBoost. After an initial search to determine a range, we will investigate more closely.

With AdaBoost, there are two types of parameters. We have parameters for AdaBoost and parameters for the weak learner it is using as a base estimator. The default base estimator is a Decision tree.

AdaBoost has the following hyperparameters.
* `n_estimators` the max number of estimators
* `learning_rate` which weights the estimators
* `algorithm` of which there are two choices

Decision trees classifiers have the following hyperparameters.
* `max_depth`
* `min_samples_split`
* `min_leaf_split`

In order to get some early results, we will break the hyperparameter grid into smaller pieces along certain axes. We will first investigate how the hyperparameters of AdaBoost impact the models and then we will tune the hyperparameters of the inner Decision tree.
from src.utils import divide_range

thresholds = divide_range(0.55,0.95,5)

base_ada_params = [{'corr_selector__threshold': thresholds,
'model__n_estimators':[int(i) for i in divide_range(20,50,5)],
'model__learning_rate': divide_range(0.5,2,5),
'model__algorithm':['SAMME', 'SAMME.R']
}]

ada_pipe = create_pipe('AdaBoost')
base_ada_results_df_v1 = get_grid_results_df(ada_pipe,
                                             'base_ada_results_df_v1', dir, 
                                             param_grid=base_ada_params,
                                             verbosity=3)

Let's see what the top scores are.
present_score_counts(base_ada_results_df_v1)
best_score = score_stats(base_ada_results_df_v1)
Let's look at the parameters associated with the top 5 scores.
from src.model_eval import collect_like_estimators

def top_n(results_df,num=5,exclude=None):
    estimators_by_score = collect_like_estimators(results_df)
    scores = list(estimators_by_score.keys())
    top = sorted(scores,reverse=True)[:num]
    for score in top:
        print(f"Score: {score}")
        present_param_counts(results_df, score, exclude)

top_n(base_ada_results_df_v1)
This gives us an idea of how to narrow down our hyperparameters.
* We will focus on a range estimators from 40-70
* We will use the algorithm `'SAMME.R'`.
* We will focus on a range of learning rate of 0.9 to 1.6
* We will focus on a range of correlation threshold from 0.77 to 0.95
thresholds = divide_range(0.77,0.95,5)

base_ada_params_v2 = [{'corr_selector__threshold': thresholds,
'model__n_estimators':[int(i) for i in divide_range(40,70,5)],
'model__learning_rate': divide_range(0.9,1.6,5),
'model__algorithm':['SAMME.R']
}]
base_ada_results_df_v2 = get_grid_results_df(ada_pipe,'base_ada_results_df_v2',
                                             dir, param_grid=base_ada_params_v2,
                                             verbosity=3)
Let's see what the top scores are.
present_score_counts(base_ada_results_df_v2)
best_score = score_stats(base_ada_results_df_v2)
Let's look at the parameters associated with the top 5 scores.
top_n(base_ada_results_df_v2,6)
It seems that an increase in the number of estimators improves performance, so we will slide the window we look at up a bit. Unfortunately, this will increase the fit time. We will take the average of the correlation thresholds. We feel we didn't cast a wide enough net with respect to learning rate, so we will look at a larger window for this parameter as well.
base_ada_params_v3 = [{'corr_selector__threshold': [0.785],
'model__n_estimators':[int(i) for i in divide_range(60,90,5)],
'model__learning_rate': divide_range(1.1,5,5),
'model__algorithm':['SAMME.R']}]

base_ada_results_df_v3 = get_grid_results_df(ada_pipe,'base_ada_results_df_v3',
                                             dir, param_grid=base_ada_params_v3,
                                             verbosity=3)
present_score_counts(base_ada_results_df_v3)
best_score = score_stats(base_ada_results_df_v3)
The scores did improve, but not significantly. If the improvement is due to values for learning rate and estimators being outside of our earlier ranges, then we will have to continue searching.
top_n(base_ada_results_df_v3)
So the number of estimators jumped a fair bit. In the case of the Logistic Regression model, we got many many high performing sets of parameters. This made it difficult to focus on a few small neighborhoods (in the parameter space) of high performing sets of parameters. Our learning rate has stabilized, so we can focus on tuning it more carefully.
base_ada_params_v4 = [{'corr_selector__threshold': [0.785],
'model__n_estimators':[int(i) for i in divide_range(75,100,5)],
'model__learning_rate': divide_range(1.05,1.15),
'model__algorithm':['SAMME.R']}]

base_ada_results_df_v4 = get_grid_results_df(ada_pipe,'base_ada_results_df_v4',
                                             dir, param_grid=base_ada_params_v4,
                                             verbosity=3)
present_score_counts(base_ada_results_df_v4)
best_score = score_stats(base_ada_results_df_v4)
top_n(base_ada_results_df_v4)
We are closing in on a learning rate of 1.1. Perhaps we need to slide our number of estimators up a bit higher. While we haven't settled on a number of estimators, we have at least settled on a learning rate. This means that the parameter grid has shrunk.
base_ada_params_v5 = [{'corr_selector__threshold': [0.785],
'model__n_estimators':[int(i) for i in divide_range(95,125,10)],
'model__learning_rate': [1.08,1.1,1.12],
'model__algorithm':['SAMME.R']}]

base_ada_results_df_v5 = get_grid_results_df(ada_pipe,'base_ada_results_df_v5',
                                             dir, param_grid=base_ada_params_v5,
                                             verbosity=3)
present_score_counts(base_ada_results_df_v5)
best_score = score_stats(base_ada_results_df_v5)
present_score_counts(base_ada_results_df_v4,2)
present_score_counts(base_ada_results_df_v5,2)
The performance is about the same. Let's look at the parameters.
standard_exclude = standard_exclude = ['model__base_estimator',
           'corr_selector__threshold','model__algorithm']
top_n(base_ada_results_df_v5,exclude=standard_exclude)

We will use 125 estimators and a learning rate of 1.12. Now to tune the Decision tree specific hyperparameters. The hyperparameters we will concern ourself with and their default values are:
* `min_samples_split`: 2
* `min_samples_leaf`: 1

from sklearn.tree import DecisionTreeClassifier


ada_params = {'corr_selector__threshold':[0.785],
            'model__n_estimators': [125],
            'model__learning_rate': [1.12], 
            'model__algorithm':['SAMME.R']}
dt_params = {'min_samples_split': [int(i) for i in divide_range(2,10)],
              'min_samples_leaf': [int(i) for i in divide_range(1,5)]}
new_dt_params = {"model__base_estimator__"+key:value 
                 for key,value in dt_params.items()}

ada_params.update(new_dt_params)
ada_params['model__base_estimator'] = [DecisionTreeClassifier(random_state=42)]
ada_dt_results_df_v1 = get_grid_results_df(ada_pipe,'ada_dt_results_df_v1',
                                             dir, param_grid=ada_params,
                                             verbosity=3)

This is performing worse than the default parameters. Which isn't so surprising since we put a cap on the 
ada_params["model__base_estimator__max_depth"] = [10]
ada_dt_results_df_v1_10 = get_grid_results_df(ada_pipe,'ada_dt_results_df_v1_10_',
                                             dir, param_grid=ada_params,
                                             verbosity=3)
present_score_counts(ada_dt_results_df_v1_5)
best_score_5 = score_stats(ada_dt_results_df_v1_5)
present_score_counts(ada_dt_results_df_v1_10)
best_score_10 = score_stats(ada_dt_results_df_v1_10)
There are many choices of parameters with the best score. Remember, we have fixed several hyperparameters already, so we will exclude them.
standard_exclude = ['model__base_estimator',
           'model__n_estimators','corr_selector__threshold',
           'model__learning_rate','model__algorithm']
exclude = standard_exclude + ['model__base_estimator__max_depth']
present_param_counts(ada_dt_results_df_v1, best_score, exclude)
It seems we have only learned that `min_samples_leaf` being 5 is a decent hyperparameter. Let's try some other values for `max_depth` and see how the results compare.
ada_params["model__base_estimator__max_depth"] = [10,20]
ada_dt_results_df_v1_10_20 = get_grid_results_df(ada_pipe,'ada_dt_results_df_v1_10_20',
                                             dir, param_grid=ada_params,
                                             verbosity=3)

present_score_counts(ada_dt_results_df_v1_10_20)
best_score = score_stats(ada_dt_results_df_v1_10_20)
The top performers in this grid search outperformed the top performers when the `max_depth` was set to 5. So we will no longer use that value (which means it will take longer to train the models).
top_n(ada_dt_results_df_v1_10_20, exclude=standard_exclude)
For the next pass, we will fix the `max_depth` at 20 and see if we can't learn anything more about what the `min_samples_leaf` and `min_samples_split` parameters should be.
ada_params['model__base_estimator__min_samples_leaf'] = [1,2,3,5]
ada_params['model__base_estimator__min_samples_split'] = [2]#4,7,10]
ada_params['model__base_estimator__max_depth'] = [20]

ada_dt_results_df_v2_20_2 = get_grid_results_df(ada_pipe,'ada_dt_results_df_v2_20_2',
                                             dir, param_grid=ada_params,
                                             verbosity=3)

ada_params['model__base_estimator__min_samples_split'] = [4]#,7,10]

ada_dt_results_df_v2_20_4 = get_grid_results_df(ada_pipe,'ada_dt_results_df_v2_20_4',
                                             dir, param_grid=ada_params,
                                             verbosity=3)

ada_params['model__base_estimator__min_samples_split'] = [7]

ada_dt_results_df_v2_20_7 = get_grid_results_df(ada_pipe,'ada_dt_results_df_v2_20_7',
                                             dir, param_grid=ada_params,
                                             verbosity=3)

ada_params['model__base_estimator__min_samples_split'] = [10]

ada_dt_results_df_v2_20_10 = get_grid_results_df(ada_pipe,'ada_dt_results_df_v2_20_10',
                                             dir, param_grid=ada_params,
                                             verbosity=3)

# min_sample_splits = 2
present_score_counts(ada_dt_results_df_v2_20_2)
best_score = score_stats(ada_dt_results_df_v2_20_2)

top_n(ada_dt_results_df_v2_20_2, exclude=exclude)
# min_sample_splits = 4
present_score_counts(ada_dt_results_df_v2_20_4)
best_score = score_stats(ada_dt_results_df_v2_20_4)

The training time is growing rapidly with the max_depth parameter of the Decision trees. We will stick to a few relatively small values for this parameter while we continue to tune the others.
In order to get intermediate results, we will train the AdaBoost model with different base estimators separately.

## Objectives

* We will investigate the data using clustering algoritthms to investigate our hypothesis # something

## Inputs

* The game data from before the train test split at the beginning of notebook 04. 

## Outputs

* A clustering model and some analysis of the underlying data set.

## Additional Comments
* This notebook follow the analysis done in notebook 07 of the Churnometer walkthrough project.
---
# Change working directory
We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()
import os

home_dir = '/workspace/pp5-ml-dashboard'
os.chdir(home_dir)
current_dir = os.getcwd()
print(current_dir)
We now load our prepared data.
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from src.utils import get_df, save_df

game_data = get_df('game_pre_split', 'datasets/clean/csv')
game_data.isna().sum(axis=1).sum()
## Section 1: Set up problem
The game of basketball as evolved over the years. For example, we saw in our exploratory data analysis that there was a correlation between 3 pointers and year. We are going to test if clustering will detect the various eras of basketball, or perhaps it will define new ones.

We will then look at the profiles of each cluster to see how it groups games of basketball and try to determine if these clusters have any correlation with time.


game_data.drop(labels=['season'],inplace=True,axis=1)
game_data.head()
Now we construct our pipeline for clustering. The format for the pipeline was inspired by the clustering pipeline in the Churnometer walkthrough project.
from sklearn.pipeline import Pipeline
from feature_engine.selection import SmartCorrelatedSelection
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

def cluster_pipe(thresh=0.7,p_components=50,clusters=50):
    pipe = Pipeline([
        ("corr_selector", SmartCorrelatedSelection(method="pearson",
                                                   threshold=thresh, 
                                                   selection_method="variance")),
        ("scaler", StandardScaler()),
        ("PCA", PCA(n_components=p_components, random_state=42)),
        ("model", KMeans(n_clusters=clusters, random_state=42)),
    ])
    return pipe
We will end up tuning the number of clusters and components as hyperparameters. It will be interesting to luck at how the analysis changes as we move between the number of clusters. At each stage, we will see how the function that assigns each game to its season behaves on clusters.

## Section 2: PCA
We start by doing some Principal component analysis.
# to suppress warnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import logging
logging.captureWarnings(True)
os.environ['PYTHONWARNINGS']='ignore'

pipe = cluster_pipe()
pca_pipe = Pipeline(pipe.steps[:-2])
game_data_pca = pca_pipe.fit_transform(game_data)

We are now going to analyze the principal components. Feel free to adjust the number of components.
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')


# This function is from the notebook 07 of the 
# churnometer walkthrough project.
def pca_component_analysis(df_pca, n_components):
    pca = PCA(n_components=n_components).fit(df_pca)

    ComponentsList = [f"Component {number}"
                      for number in range(n_components)]
    dfExplVarRatio = pd.DataFrame(
        data=np.round(100 * pca.explained_variance_ratio_, 3),
        index=ComponentsList,
        columns=['Explained Variance Ratio (%)'])

    dfExplVarRatio['Accumulated Variance'] = dfExplVarRatio['Explained Variance Ratio (%)'].cumsum(
    )
    PercentageOfDataExplained = dfExplVarRatio['Explained Variance Ratio (%)'].sum(
    )
    print(
        f"* The {n_components} components explain {round(PercentageOfDataExplained,2)}% of the data \n")
    plt.figure(figsize=(9, 6))
    sns.lineplot(data=dfExplVarRatio,  marker="o")
    plt.xticks(rotation=90)
    plt.yticks(np.arange(0, 110, 10))
    plt.show()


pca_component_analysis(game_data_pca, 15)

As we are hoping the data will be grouped into eras, each of which is multiple years long. We expect there to be a fair number of clusters, so we don't mind having many components.
pca_component_analysis(game_data_pca,9)

So we redefine our pipeline creation function to have 9 as the number of components.
def cluster_pipe(thresh=0.7,clusters=50):
    pipe = Pipeline([
        ("corr_selector", SmartCorrelatedSelection(method="pearson",
                                                   threshold=thresh, 
                                                   selection_method="variance")),
        ("scaler", StandardScaler()),
        ("PCA", PCA(n_components=9, random_state=42)),
        ("model", KMeans(n_clusters=clusters, random_state=42)),
    ])
    return pipe
Now we need to use the elbow method and look at silhouette scores.

## Section 3: Elbow Method and Silhouette scores
These will help us determine the appropriate number of clusters to use for our algorithm. We are also taking into account some domain knowledge, which is that eras in basketball range from 6 to 10 years. It is not a well defined concept so we don't have a hard number. Our data set ranges from 1985 until 2022. So we (conservatively) expect there to be between 4 and 7 eras.
new_pipe = cluster_pipe()
pca_part_of_pipe = Pipeline(new_pipe.steps[:-1])
game_data_pca = pca_part_of_pipe.fit_transform(game_data)
from yellowbrick.cluster import KElbowVisualizer

visualizer = KElbowVisualizer(KMeans(random_state=42), k=(1,11))
visualizer.fit(game_data_pca) 
visualizer.show() 
plt.show()
They are suggesting 4 clusters, but 6 also looks promising. Let's see how the Silhouette scores behave with respect to these different numbers of clusters.
from yellowbrick.cluster import SilhouetteVisualizer

n_cluster_start = 3
n_cluster_stop = 8

print("=== Average Silhouette Score for different number of clusters ===")
visualizer = KElbowVisualizer(KMeans(random_state=42), k=(
    n_cluster_start, n_cluster_stop), metric='silhouette')
visualizer.fit(game_data_pca)
visualizer.show()
plt.show()
print("\n")


for n_clusters in np.arange(start=n_cluster_start, stop=n_cluster_stop):

    print(f"=== Silhouette plot for {n_clusters} Clusters ===")
    visualizer = SilhouetteVisualizer(estimator=KMeans(n_clusters=n_clusters,
                                                       random_state=42),
                                      colors='yellowbrick')
    visualizer.fit(game_data_pca)
    visualizer.show()
    plt.show()
    print("\n")


The analysis suggest using 3 clusters. After adding the clustering data to the data frame we will see if the clustering correlates with the `season` feature.
pipe_w_clusters = cluster_pipe(clusters=3)
game_w_clusters = game_data.copy()
game_w_clusters.head()
Now we will fit both of these pipelines and see what they tell us.
pipe_w_clusters.fit(game_w_clusters)
# this can be skipped and done after we have the profile of the clusters
game_data_w_seasons = get_df('game_pre_split', 'datasets/clean/csv')
def get_season(game_id):
    return game_data_w_seasons.loc[game_id, 'season']
    # ai suggested the above but I think it should be this one
    #return game_data_w_seasons.loc[game_id]['season']
game_test = game_w_clusters.copy()

game_test.head()
game_w_clusters['Clusters'] = pipe_w_clusters.predict(game_data)

game_w_clusters.head()
print(f"* Cluster frequencies \n{game_w_clusters['Clusters'].value_counts(normalize=True).to_frame().round(2)} \n\n")
game_w_clusters['Clusters'].value_counts().sort_values().plot(kind='bar')
plt.show()
It is interesting to see how the clusters behave with respect to the components from the PCA step.
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")
def compare_components_w_clusters(a:int,b:int,pca_step,cluster_step,pipe):
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=pca_step[:, a], y=pca_step[:, b],
                hue=cluster_step['Clusters'], palette='Set1', alpha=0.6)
    plt.scatter(x=pipe['model'].cluster_centers_[:, 0],
            y=pipe['model'].cluster_centers_[:, 1],
            marker="x", s=169, linewidths=3, color="black")
    plt.xlabel(f"PCA Component {a}")
    plt.ylabel(f"PCA Component {b}")
    plt.title("PCA Components colored by Clusters")
    plt.show()

pairs = [(a,b) for a in range(9) for b in range(9) if a<b]
print(len(pairs))



Let's investigate how the clusters and the components from the PCA relate to one another.
import seaborn as sns
import matplotlib.pyplot as plt


sns.set_style("whitegrid")


def compare_components_w_clusters(a:int,b:int,pca_step,cluster_step,pipe):
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=pca_step[:, a], y=pca_step[:, b],
                hue=cluster_step['Clusters'], palette='Set1', alpha=0.6)
    plt.scatter(x=pipe['model'].cluster_centers_[:, 0],
            y=pipe['model'].cluster_centers_[:, 1],
            marker="x", s=169, linewidths=3, color="black")
    plt.xlabel(f"PCA Component {a}")
    plt.ylabel(f"PCA Component {b}")
    plt.title("PCA Components colored by Clusters")
    plt.show()

interesting_pairs = [(0,1), (0,3), (6,8)]
for a,b in interesting_pairs:
    compare_components_w_clusters(a, b, game_data_pca, game_w_clusters,
                                  pipe_w_clusters)

Feel free to look at how other components compare with respect to the clusters. We felt these three were the most interesting.

## Cluster Profile
We next wish to determine the profile of these clusters. We do this by training a classification model on the data with the clusters as the target. The important features of the models will help us to determine the profiles of the individual clusters.

In our model selection notebook, we found that AdaBoost and Logistic Regression were good models for working with this data. We will use AdaBoost as it works one multi-class classification problems without any need for adjustment.
# This will be our target variable.
cluster_predictions = game_w_clusters['Clusters']

# This will be our data.
df = game_w_clusters.copy()
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
                    df.drop(['Clusters'], axis=1),
                    df['Clusters'], test_size=0.2,
                    random_state=42)

print(X_train.shape)
print(X_test.shape)
Now we create our classifier pipeline. We will try it with the default parameters as well as the parameters we found in the last notebook.
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_selection import SelectFromModel


def clf_pipe(thresh=0.7, params={}):
    pipe = Pipeline([
        ('corr_selector',
         SmartCorrelatedSelection(method="pearson",
                                  threshold=thresh,
                                  selection_method="variance"))
                        ])
    pipe.fit(X_train)
    dropping = pipe['corr_selector'].features_to_drop_
    new_assignments = { key: [val for val in value if val not in dropping] 
                       for key,value in TRANSFORM_ASSIGNMENT.items()}
    for transform, targets in new_assignments.items():
        if not targets:
            continue
        pipe.steps.append(
            (transform, TRANSFORMS[transform][0](variables=targets))
            )
    pipe.steps.append(('scaler', StandardScaler()))
    model = AdaBoostClassifier(random_state=42,**params)
    pipe.steps.append(("feat_selection", SelectFromModel(model)))
    pipe.steps.append(('model',model))
    return pipe

ada_thresh = 0.8
ada_params = {'n_estimators': 110, 'learning_rate': 1.133,
              'algorithm': 'SAMME.R'}
default_ada_pipe = clf_pipe()
default_ada_pipe.fit(X_train, y_train)
tuned_ada_pipe = clf_pipe(thresh=ada_thresh, params=ada_params)
tuned_ada_pipe.fit(X_train, y_train)


It is interesting that attempted 3 points is more important to the clustering than attempted shots of the away team. We will use some functions to get the profile of the clusters from the model we trained. We will use the following functions to determine the profile of the clusters. They are all from the Chrunometer walkthrough project.
# All functions are from notebook 07 of the Churnometer walkthrough
def DescriptionAllClusters(df, decimal_points=3):

    DescriptionAllClusters = pd.DataFrame(
        columns=df.drop(['Clusters'], axis=1).columns)
    # iterate on each cluster , calls Clusters_IndividualDescription()
    for cluster in df.sort_values(by='Clusters')['Clusters'].unique():

        EDA_ClusterSubset = df.query(
            f"Clusters == {cluster}").drop(['Clusters'], axis=1)
        ClusterDescription = Clusters_IndividualDescription(
            EDA_ClusterSubset, cluster, decimal_points)
        DescriptionAllClusters = DescriptionAllClusters.append(
            ClusterDescription)

    DescriptionAllClusters.set_index(['Cluster'], inplace=True)
    return DescriptionAllClusters


def Clusters_IndividualDescription(EDA_Cluster, cluster, decimal_points):
    ClustersDescription = pd.DataFrame(columns=EDA_Cluster.columns)
    # for a given cluster, iterate over all columns
    # if the variable is numerical, calculate the IQR: display as Q1 -- Q3.
    # That will show the range for the most common values for the numerical variable
    # if the variable is categorical, count the frequencies and displays the top 3 most frequent
    # That will show the most common levels for the category

    for col in EDA_Cluster.columns:
        try:  # eventually a given cluster will have only missing data for a given variable
            if EDA_Cluster[col].dtypes in ['float', 'int']:
                DescStats = EDA_Cluster.dropna(subset=[col])[[col]].describe()
                Q1 = round(DescStats.iloc[4, 0], decimal_points)
                Q3 = round(DescStats.iloc[6, 0], decimal_points)
                Description = f"{Q1} -- {Q3}"
                ClustersDescription.at[0, col] = Description
            else:
                raise ValueError(f"Wrong data type for {col}: {EDA_Cluster[col].dtypes}.")
        except Exception as e:
            ClustersDescription.at[0, col] = 'Not available'
            print(
                f"** Error Exception: {e} - cluster {cluster}, variable {col}")
    ClustersDescription['Cluster'] = str(cluster)
    return ClustersDescription


import plotly.express as px


def cluster_distribution_per_variable(df, target):
    """
    The data should have 2 variables, the cluster predictions and
    the variable you want to analyze with, in this case we call "target".
    We use plotly express to create 2 plots:
    Cluster distribution across the target.
    Relative presence of the target level in each cluster.
    """
    df_bar_plot = df.value_counts(["Clusters", target]).reset_index()
    df_bar_plot.columns = ['Clusters', target, 'Count']
    df_bar_plot[target] = df_bar_plot[target].astype('object')

    print(f"Clusters distribution across {target} levels")
    fig = px.bar(df_bar_plot, x='Clusters', y='Count',
                 color=target, width=800, height=500)
    fig.update_layout(xaxis=dict(tickmode='array',
                      tickvals=df['Clusters'].unique()))
    fig.show(renderer='jupyterlab')

    df_relative = (df
                   .groupby(["Clusters", target])
                   .size()
                   .groupby(level=0)
                   .apply(lambda x:  100*x / x.sum())
                   .reset_index()
                   .sort_values(by=['Clusters'])
                   )
    df_relative.columns = ['Clusters', target, 'Relative Percentage (%)']

    print(f"Relative Percentage (%) of {target} in each cluster")
    fig = px.line(df_relative, x='Clusters', y='Relative Percentage (%)',
                  color=target, width=800, height=500)
    fig.update_layout(xaxis=dict(tickmode='array',
                      tickvals=df['Clusters'].unique()))
    fig.update_traces(mode='markers+lines')
    fig.show(renderer='jupyterlab')

df_cluster_profile = df.copy()
df_cluster_profile = df_cluster_profile.filter(items=best_features + ['Clusters'], axis=1)

pd.set_option('display.max_colwidth', None)
clusters_profile = DescriptionAllClusters(df_cluster_profile, decimal_points=0)
clusters_profile
There are so many different relevant features. We think graphing the distributions may be more illuminating.
feature_pairs = []
feature_stems = list({term.split('_')[0] for term in best_features})
for stem in feature_stems:
    home_stem = stem + '_home'
    away_stem = stem + '_away'
    if home_stem in best_features and away_stem in best_features:
        feature_pairs.append((home_stem, away_stem))
    elif home_stem in best_features:
        feature_pairs.append((home_stem, ''))
    elif away_stem in best_features:
        feature_pairs.append(('',away_stem))

for home, away in feature_pairs:
    if home and away:
        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
        sns.histplot(data=df, x=home, kde=True, element="step", ax=axes[0],
                     hue='Clusters',palette='deep')
        sns.histplot(data=df, x=away, kde=True, element="step", ax=axes[1],
                     hue='Clusters', palette='deep')
        axes[0].set_title(f'{home}')
        axes[1].set_title(f'{away}')
        plt.show()
    elif home:
        fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))
        sns.histplot(data=df, x=home, kde=True, element="step", ax=axes,
                     hue='Clusters', palette='deep')
        axes.set_title(f'{home}')
        plt.show()
    elif away:
        fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))
        sns.histplot(data=df, x=away, kde=True, element="step", ax=axes,
                     hue='Clusters', palette='deep')
        axes.set_title(f'{away}')
        plt.show()
## Refit Clustering
We now refit the clustering pipeline with the most relevant features determined by the profile in the last section. This will lead to a more focused model since it is only being trained on the features already determined to be most relevant to the clustering.
to_drop = [col for col in game_data.columns if col not in best_features]
new_assignments = { key: [val for val in value if val not in to_drop] 
                       for key,value in TRANSFORM_ASSIGNMENT.items()}
    
def last_cluster_pipe(clusters=3):
    pipe = Pipeline([('yeo_johnson',vt.YeoJohnsonTransformer(
                                    variables=new_assignments['yeo_johnson'])),
                                    ('box_cox',vt.BoxCoxTransformer(
                                    variables=new_assignments['box_cox'])),
                                    ('scaler', StandardScaler()),
                                    ('model', KMeans(n_clusters=3, 
                                                     random_state=42))])
    return pipe

game_data_reduced= game_data.copy().filter(best_features)
game_data_reduced.head()

We redo the Elbow method and Silhoutte score analysis with this modified pipeline. Again, we are following the Churnometer project quite closely.
cluster_pipe = last_cluster_pipe()
pipeline_analysis = Pipeline(cluster_pipe.steps[:-1])
df_analysis = pipeline_analysis.fit_transform(game_data_reduced)
from yellowbrick.cluster import KElbowVisualizer
visualizer = KElbowVisualizer(KMeans(random_state=0), k=(1,11))
visualizer.fit(df_analysis) 
visualizer.show() 
plt.show()
from yellowbrick.cluster import SilhouetteVisualizer

n_cluster_start, n_cluster_stop = 2, 7

print("=== Average Silhouette Score for different number of clusters ===")
visualizer = KElbowVisualizer(KMeans(random_state=0), k=(
    n_cluster_start, n_cluster_stop), metric='silhouette')
visualizer.fit(df_analysis)
visualizer.show()
plt.show()
print("\n")


for n_clusters in np.arange(start=n_cluster_start, stop=n_cluster_stop):

    print(f"=== Silhouette plot for {n_clusters} Clusters ===")
    visualizer = SilhouetteVisualizer(estimator=KMeans(n_clusters=n_clusters, random_state=0),
                                      colors='yellowbrick')
    visualizer.fit(df_analysis)
    visualizer.show()
    plt.show()
    print("\n")

Again, 3 clusters gives the highest Silhoutte coefficient.
new_game_w_clusters = game_data_reduced.copy()
last_pipe = last_cluster_pipe(clusters=3)
last_pipe.fit(new_game_w_clusters)
new_game_w_clusters['Clusters'] = last_pipe.predict(new_game_w_clusters)
new_game_w_clusters.head()

print(f"* Cluster frequencies \n{new_game_w_clusters['Clusters'].value_counts(normalize=True).to_frame().round(2)} \n\n")
new_game_w_clusters['Clusters'].value_counts().sort_values().plot(kind='bar')
plt.show()

game_seasons = get_df('game_pre_split', 'datasets/clean/csv').filter(['game_id','season'])
game_seasons.set_index('game_id',inplace=True)
new_clusters_only = new_game_w_clusters.filter(['Clusters'])
new_season_clusters = new_clusters_only.join(game_seasons)


new_cluster_season_corr = new_season_clusters.corr()
print(new_cluster_season_corr.head())

Lets look at the distribution of seasons for each cluster like we did previously. We will compare it side by side with the old distribution. Note, the labels of the clusters may have changed so we have regruoped them by trying to match distributions.
pairs = [(0,2),(1,0),(2,1)]
for i,j in pairs:
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
    old_cluster = season_clusters.query(f'Clusters == {i}')
    new_cluster = new_season_clusters.query(f'Clusters == {j}')
    sns.countplot(data=old_cluster, x="season", ax=ax[0]).set_title(f"Old Cluster {i}")
    sns.countplot(data=new_cluster, x="season", ax=ax[1]).set_title(f"New Cluster {j}")
    plt.show()



So perhaps the clusters are related to time. Let's look at the other correlations with respect to the clustering.
clusters_correlation = game_w_clusters.corr()
print(clusters_correlation['Clusters'].sort_values(ascending=False)[:8])
We will see this again when we attempt to classify the clusters using an adaptive boost model.
Let's investigate how the clusters and the components from the PCA relate to one another.